# LoadinG The Data

In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import math

#using the preprocess Sqlite Table to read the  data
con = sqlite3.connect('./amazon-fine-food-reviews/finalsqlite')

#Filtering only positive review
positive_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'positive'
""", con)

#Extracting 2000 Positive review Randomly
positive_smaller_review_set = positive_review.sample(50)
print(positive_smaller_review_set.shape)

#Filtering only negative review
negative_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'negative'
""", con)

#Extracing 2000 random negative review
negative_smaller_review_set = negative_review.sample(50)
print(negative_smaller_review_set.shape)

#Concanating Negative and Positive review into one dataframe
frames = [positive_smaller_review_set, negative_smaller_review_set]
final = pd.concat(frames)

#Sort the dataframe data into the increasing Time order
final = final.sort_values(by=['Time'])
print(final.head)
print(final.shape)


C:\Users\DANAV\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(50, 12)
(50, 12)
<bound method NDFrame.head of          index      Id   ProductId          UserId  \
21125   510894  552406  B0007OPW5C  A393SW83L4WH3G   
16501    63064   68516  B0005YWSYM  A1IL6W1NK05UW9   
7834    125523  136190  B000EPOC1Y  A1WX42M589VAMQ   
73503   395103  427242  B000HZLPRU  A22I82BMS8S5UJ   
73894   135839  147428  B000I4PXBO   AUUMSYICW2I0K   
10235   120889  131052  B0001LW132  A158HXAKJ150LZ   
12061   296733  321431  B000HDKZJQ  A2KR0I0OG4OI0D   
50166    72612   79032  B000F1X740   ABAPF3XY1W6JX   
102471   80533   87562  B000SV7B76   AXSH8IKKHRWRW   
1511    376805  407450  B000084F04  A3J0OXB9KIC5SS   
5689    475816  514550  B000CQ6KTM  A3B7QUFEZ9IMBK   
19381   499672  540244  B0006SDRRE   AF3CAM6HWRGKY   
30724    15321   16727  B001LGGH40  A2CIQEY05SPHTZ   
30659     6196    6732  B001LG945O  A37USOKUTI724H   
17672   493570  533627  B000SATIWI  A2MLCH9REYK6Q7   
160956    8718    9550  B001EQ5IPQ   A11V7W73DYY0A   
70582   460225  497702  B000HDK0D2

In [25]:
# Function to  find accuracy and optimal K(nearest Neighbor) on given approach and given vector

def knn_accuracy(standarized_data , algorithms):
    # create design matrix X and target vector y
    X = np.array(standarized_data[:,:])  
    y = np.array(final['Score'].values)

    #Calculating the train and test size and dividing the data into train and test chunks into the 70 - 30 ratio
    train_size = math.floor(X.shape[0] * 0.7)

    #split the data set into train and test
    X_train = X[: train_size ,:]
    X_test  = X[train_size : , :]  
    y_train = y[: train_size ]
    y_test =  y[train_size : ]


    neighbors = np.arange(1, 30 , 2)

    # empty list that will hold cv scores
    cv_scores = []
    print(algorithms)
     
    # perform 10-fold cross validation
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k,  algorithm = algorithms )
        scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    # changing to misclassification error
    MSE = [1 - x for x in cv_scores]
    print(MSE)
    # determining best k
    optimal_k = neighbors[MSE.index(min(MSE))]
    print('\nThe optimal number of neighbors is %d.' % optimal_k)


    #calculate the accuracy on Test data

    knn = KNeighborsClassifier(n_neighbors = optimal_k , algorithm = algorithms)

    # fitting the model
    knn.fit(X_train, y_train)

    #predict the response
    pred = knn.predict(X_test)

    # evaluate accuracy
    acc = accuracy_score(y_test, pred) * 100
    
    return [optimal_k, acc]


# Bag OF word(BOW)

In [26]:
#BOW of preprocess text
count_vect = CountVectorizer()
final_counts_preprocess_text = count_vect.fit_transform(final['CleanedText'].values)

print("Shape of vector of Preprocess review(text) after BOW(unigram)",final_counts_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after BOW(unigram) (100, 1737)


# Standariztion Of Vector

In [27]:
from sklearn.preprocessing import StandardScaler

#Standarization OF review text
standarized_data = StandardScaler().fit_transform(final_counts_preprocess_text.toarray()) 

C:\Users\DANAV\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


 # knn on BOW vector

In [28]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on BOW using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on BOW using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5017857142857143, 0.5142857142857142, 0.5392857142857143, 0.5101190476190476, 0.44166666666666665, 0.4750000000000001, 0.4833333333333334, 0.475, 0.444047619047619, 0.46071428571428574, 0.4642857142857143, 0.5142857142857142, 0.5351190476190476]

The optimal number of neighbors is 13.

The accuracy of the knn classifier on BOW using Brute Force approach for k = 13 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5017857142857143, 0.5142857142857142, 0.5392857142857143, 0.5101190476190476, 0.44166666666666665, 0.4750000000000001, 0.4833333333333334, 0.475, 0.444047619047619, 0.46071428571428574, 0.4642857142857143, 0.5142857142857142, 0.5351190476190476]

The optimal number of neighbors is 13.

The accuracy of the knn classifier on BOW using K_D tree approach for k = 13 is 53.333333%


# Tf-Idf

In [11]:
#tf-idf of preprocess text
tf_idf_vect  = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf_preprocess_text =  tf_idf_vect.fit_transform(final['Text'].values)

#Shape of  vectors
print("Shape of vector of Preprocess review(text) after Tf-Idf",final_tf_idf_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after Tf-Idf (100, 7274)


# Standariztion Of Vector

In [12]:
from sklearn.preprocessing import StandardScaler

#Standarization OF review text
standarized_data = StandardScaler().fit_transform(final_tf_idf_preprocess_text.toarray())

# knn on TF-IDF vector 

In [13]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Tf-Idf using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Tf-Idf using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using K_D tree approach for k = 21 is 53.333333%


# Average Word2Vec

In [17]:
#Definition oF CleanHtml and CleanPunc for Preprocessing the reviw text
import re
import gensim

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

# Train your own Word2Vec model using your own text corpus

i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
    
   

In [18]:
# Train the model
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=1,size=50, workers=4)
                                 
Avg_sent_vectors = []
for sent in list_of_sent:
    sent_vect = np.zeros(50)
    count = 0
    for word in sent:
        try :
            sent_vect += w2v_model.wv[word]
            count = count + 1
        except:
            pass
    
    sent_vect = sent_vect / count
    Avg_sent_vectors.append(sent_vect)
     
#Standarization Of Avg Word2Vec vector
standardized_data = StandardScaler().fit_transform(Avg_sent_vectors)
print(standardized_data.shape)



(100, 50)


# knn on Average Word2Vec vector 

In [19]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Word2vec using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Word2vec using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Word2vec using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Word2vec using K_D tree approach for k = 21 is 53.333333%


# Tf-Idf Word2vec

In [21]:
tf_idf_feat = tf_idf_vect.get_feature_names()
tfidf_sent_vect=[]
row= 0;
for sent in list_of_sent:
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        try:
            if((word in  w2v_model.wv.vocab) and (word in tf_idf_feat)):
                w_vec = w2v_model.wv[word]
                tf_idf_vec = final_tf_idf[row,tf_idf_feat.index(word)]
                sent_vec = sent_vec + (w_vec*tf_idf_vec)
                weight_sum = weight_sum + tf_idf_vec
        except:
            pass
    sent_vec = sent_vec/weight_sum
    tfidf_sent_vect.append(sent_vec)
    row= row+1
    
# Removing NAN values    
tfidf_sent_vect_values = np.nan_to_num(tfidf_sent_vect)

#Standarization Of Tf-Idf Word2Vec Vector
standardized_data = StandardScaler().fit_transform(tfidf_sent_vect_values)
print(standardized_data.shape)



C:\Users\DANAV\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


(100, 50)


# knn on Tf-Idf Word2Vec vector 

In [22]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Tf-Idf Word2vec using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Tf-Idf Word2vec using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf Word2vec using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf Word2vec using K_D tree approach for k = 21 is 53.333333%
